In [ ]:
!pip install transformers 
!pip install sentencepiece

# NLG For Conversational Q&A
**Goal:**

Goal was to assemble a NLP pipeline which is able to answer a question and providing a longer text response (usually one whole sentence) which uses words used in question for conversational closed domain q&a. 

**Approach:**

1. pretrained BERT Q&A model for question answer answering which generates short answers (keywords)
2. finetune pretrained T5 model to generate longer answers which also takes the test of the question into consideration aswell as the keyword answers to suit a conversational environment

**How To Finetune T5?**

Finetuning via few shot learning. Meaning T5 is able to learn a new task with few training data because T5 has already knowledge about the language. Training data example: `q: Bis wann muss ich meine Wohnung nach Einzug anmelden? a: innerhalb von 14 Tagen` -> `Sie müssen Ihre Wohnung innerhalb von 14 Tagen anmelden.`


**Results & Findings**
* BERT's ability to find answers is acceptable. BERT is not able to find more complex answers, especially involving logical functions like AND or OR. 
* Even though T5 was finetuned only on 20 domain examples it does generate good quality responses for unseen data. The Problem is that the best generated response is not always the highest ranked text thus a human is still needed to pick the best response. One way to fix it is to probably train it with more data.
* Practical implementation and integration in Rasa: high RAM requirements to load two relative large models -> probably does not run on computers with less than 8GB of RAM together with the rest of Rasa; high interference time (5-6s per question without Rasa NLU and without GPU acceleration) -> less suitable for live chat


## Finetune T5 via Few Shot Learning


In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [3]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [4]:
df = pd.read_csv('./t5-train.csv', sep=';')

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
t5_model.to(device)

In [ ]:
t5_model.train()

epochs = 100

for epoch in range(epochs):
  print ("epoch: ", epoch)
  for index, row in df.iterrows():
    input_sent = "q: " + row['question'] + " a: " + row['short']
    ouput_sent = row['long']

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")

    input_ids  = tokenized_inp["input_ids"].to(device)
    attention_mask = tokenized_inp["attention_mask"].to(device)

    lm_labels= tokenized_output["input_ids"].to(device)
    decoder_attention_mask=  tokenized_output["attention_mask"].to(device)

    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [7]:
t5_model.save_pretrained("./t5/", push_to_hub=False)

## Q&A NLG Pipeline 

In [1]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration 

In [2]:
qa_pipeline = pipeline(
    "question-answering",
    model="Sahajtomar/GBERTQnA",
    tokenizer="Sahajtomar/GBERTQnA"
)
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("./t5/")

In [19]:
context="Mit einem Führungszeugnis können Sie nachweisen, dass Sie nicht vorbestraft sind. Führungszeugnisse unterscheidet man danach, ob sie bestimmt sind für private Zwecke (zum Beispiel für Ihren Arbeitgeber) oder für Behörden (sogenanntes „behördliches Führungszeugnis“, auch „Führungszeugnis zur Vorlage bei einer Behörde“). Außerdem gibt es unterschiedliche Arten von Führungszeugnissen nämlich, ein einfaches Führungszeugnis und ein erweitertes Führungszeugnis. Angehörige anderer EU-Staaten erhalten ein europäisches Führungszeugnis. Europäische Führungszeugnisse enthalten auch Strafregister-Einträge aus Ihrem Heimatland. Das Führungszeugnis wird erstellt vom Bundesamt für Justiz in Bonn (Bundeszentralregister). Wird das Führungszeugnis für private Zwecke benötigt, erhalten Sie es postalisch an Ihre Anschrift übersandt; eines für behördliche Zwecke geht direkt an die Behörde."
question="Was stehet in dem Europäischen Führungszeugnis?"
question

'Was stehet in dem Europäischen Führungszeugnis?'

In [20]:
qa_res = qa_pipeline({'context': context, 'question': question})['answer']
qa_res

'Strafregister-Einträge'

In [22]:
input = "q: " + question + " a: " + qa_res
tokenized = tokenizer.encode_plus(input, return_tensors="pt")

input_ids = tokenized["input_ids"]
attention_mask = tokenized["attention_mask"]

model.eval()
beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print (sent)

Der Europäische Führungszeugnis wird durch Strafregister-Einträge ersetzt.
Das Europäische Führungszeugnis wird vom Strafregister-Einträge begleitet.
Das Europäische Führungszeugnis beinhaltet Ihre Strafregister-Einträge.
Das Europäische Führungszeugnis wird durch Strafregister-Einträge ersetzt.
Der Europäische Führungszeugnis wird vom Strafregister-Einträge begleitet.
